# Slowly Changing Dimensions (SCD)

Slowly Changing Dimensions (SCD) are a crucial concept in dimension modeling, particularly in the context of data warehousing and business intelligence. SCD deals with how to handle changes to dimension attributes over time, ensuring that historical data remains accurate and available for analysis. There are typically three common types of Slowly Changing Dimensions:

1. **Type 0: retain original**

   The Type 0 dimension attributes never change and are assigned to attributes that have durable values or are described as 'Original'. Examples: Date of Birth, Original Credit Score. Type 0 applies to most date dimension attributes



2. **Type 1: No History (Overwrite)**:
   - In this approach, when a dimension attribute changes, the old value is simply overwritten with the new value.
   - Historical data is not preserved, and the dimension reflects only the most recent state.
   - This method is suitable when historical data is not important for analysis, or when storage efficiency is a primary concern.
   - **Example:**

| CustomerID |   Name   |   City    |     email        |   ......     |
|------------|----------|-----------|------------------|--------------|
|     1      |   Adam   |  New York | adam@example.com |   ......     |

   Lets suppose that Adam moves from New York to New Jersey. 

| CustomerID |     Name     |     City     |        email        |     .......   |
|------------|--------------|--------------|---------------------|---------------|
|     1      |       Adam   |   New Jersey |   adam@example.com  |               |

   So we don't have any information about the previous city.


**SQL COde:**
```
CREATE TABLE customer (
   customer_id  INT,
   customer_name VARCHAR(50),
   customer_email VARCHAR(50),
   customer_phone VARCHAR(15),
   load_date Date,
   customer_address VARCHAR(255)
);

INSERT INTO customer VALUES
   (1, 'John Doe', 'John.doe@example.com', '123-456-7890', '2022-01-01', '123 Main St')
   (2, 'Jane Doe', 'jane.doe@example.com', '987-856-8795', '2022-01-01', '456 Elm St')
   (1, 'Bob Smith', 'bob.smith@example.com', '123-456-7890', '2022-01-01', '789 Oak St');

UPDATE customer SET customer_address = '789 Maple St' WHERE customer_id = 2;
```


3. **Type 2: Add New Row (Historical)**:
- In the Type 2 SCD approach, when a dimension attribute changes, a new row is added to the dimension table to capture the new attribute value.
- This preserves the historical state of the dimension, allowing for historical analysis.
- Each row typically has an effective date range to indicate when it was valid.
- A surrogate key is often used to relate fact tables to the dimension tables.
- Type 2 is appropriate when historical data is important for analysis and reporting.
- This kind 
- **Example: (Using a flag)** In thia approach, we use a flag to indicate if a particular value is active or if it is current.
 

|   SurrogateID   | CustomerID |   Name   |   City    |     email        |    isActive   |   ......     |
|-----------------|------------|----------|-----------|------------------|---------------|--------------|
|        1        |     1      |   Adam   |  New York | adam@example.com |     True      |   ......     |

A flag is added (`isActive`). 

|   SurrogateID   | CustomerID |   Name   |   City        |     email        |    isActive    |   ......     |
|-----------------|------------|----------|---------------|------------------|----------------|--------------|
|        1        |     1      |   Adam   |  New York     | adam@example.com |     False      |   ......     |
|        2        |     1      |   Adam   |  New Jersey   | adam@example.com |     False      |   ......     |
|        3        |     1      |   Adam   |  Miami        | adam@example.com |     True       |   ......     |

   To get latest update on all customer, or a given customer, we just need to query `isActive=True`. Everytime there is a change, we need to add a new row and update the `isActive` column of the previous rows to `False` and newest to `True`.

   - **Example: (using version numbers)**

|   SurrogateID   | CustomerID |   Name   |   City    |     email        |    Version    |   ......     |
|-----------------|------------|----------|-----------|------------------|---------------|--------------|
|        1        |     1      |   Adam   |  New York | adam@example.com |        0      |   ......     |

Here a version number is used. 

|   SurrogateID   | CustomerID |   Name   |   City        |     email        |    Version     |   ......     |
|-----------------|------------|----------|---------------|------------------|----------------|--------------|
|        1        |     1      |   Adam   |  New York     | adam@example.com |     0          |   ......     |
|        2        |     1      |   Adam   |  New Jersey   | adam@example.com |     1          |   ......     |
|        3        |     1      |   Adam   |  Miami        | adam@example.com |     2          |   ......     |

   Here we need to filter on the `max(Version)` column to get the recent values.

   - **Example: (using date ranges)** In this approach, we use date ranges to show the period a particular record (row) was active:

|   SurrogateID   | CustomerID |   Name   |   City    |     email        |    StartDate      |    EndDate      |   ......     |
|-----------------|------------|----------|-----------|------------------|-------------------|-----------------|--------------|
|        1        |     1      |   Adam   |  New York | adam@example.com |    01-Jan-2020    |      NULL       |   ......     |

Here a version number is used. 

|   SurrogateID   | CustomerID |   Name   |   City       |     email        |    StartDate      |    EndDate      |   ......     |
|-----------------|------------|----------|--------------|------------------|-------------------|-----------------|--------------|
|        1        |     1      |   Adam   |  New York    | adam@example.com |    01-Jan-2020    |   25-Mar-2020   |   ......     |
|        2        |     1      |   Adam   |  New Jersey  | adam@example.com |    25-Mar-2020    |   01-Dec-2020   |   ......     |
|        3        |     1      |   Adam   |  Miami       | adam@example.com |    01-Dec-2020    |      NULL       |   ......     |

   - **Example: (Using Flag and Date)** In this approach, we use date ranges and Flag both to show the period a particular record (row) was active:

|   SurrogateID   | CustomerID |   Name   |   City       |     email        |    StartDate      |    EndDate      |   isActive   |   ......     |
|-----------------|------------|----------|--------------|------------------|-------------------|-----------------|--------------|--------------|
|        1        |     1      |   Adam   |  New York    | adam@example.com |    01-Jan-2020    |   25-Mar-2020   |   False     |   ......      |
|        2        |     1      |   Adam   |  New Jersey  | adam@example.com |    25-Mar-2020    |   01-Dec-2020   |   False     |   ......      |
|        3        |     1      |   Adam   |  Miami       | adam@example.com |    01-Dec-2020    |      NULL       |   True     |   ......       |

```
ALTER TABLE customer ADD COLUMN customer_segment VARCHAR(20);
ALTER TABLE customer ADD COLUMN start_date DATE;
ALTER TABLE customer ADD COLUMN end_date DATE;
ALTER TABLE customer ADD COLUMN version BIGINT DEFAULT;

UPDATE customer SET customer_segment = 'GOLD', start_date = '2022-02-01', end_date = '9999-12-31', WHERE customer_id = 2;
```

4. **Type 3: Add Columns (Partial Historical)**:
   - Type 3 SCD involves adding new columns to the dimension table to store a limited amount of historical data. Typically, two columns are added: one for the current value and one for the previous value.
   - This approach allows for limited historical analysis by comparing the current and previous values.
   - It's useful when full historical data is not needed, and there is a desire to keep historical changes in the same row.

| CustomerID |   Name   |   City         |      PrevCity    | email           |     ......    |
|------------|----------|----------------|------------------|-----------------|---------------|
|     1      |   Adam   |  New Jersey    |     New York     |adam@example.com |    ......     |

   So in this case, we only can retain infomration upto one step earlier. 

5. **Type 4: Hybrid Slowly Changing Dimension (SCD)**:
   - Also known as "Type 4" or "SCD Type 4," this hybrid approach combines elements of both Type 2 and Type 1.
   - In this method, a separate historical table is created to track changes over time while the main dimension table remains static.
   - The historical table captures the history of changes, allowing for historical analysis.
   - It can be more space-efficient compared to a pure Type 2 approach since the main dimension table remains relatively small.
   - **Example:** In the following diagram, if we assume that the carpool (also known as High occupancy vehicles) pass needs to be purchased everymonth, we can move that field to a smaller mini-dimesnion and reference it directly from the fact table

<img src="images/scd-41.png" alt="Types of data" style="max-width: 800px;"/> 

   we can split the table into a mini DimCarPool Dimension.

<img src="images/scd-42.png" alt="Types of data" style="max-width: 800px;"/> 


3. **Type 6: Type 1 + Type 2 (Combination SCD)**:
   - Type 6 SCD is a combination of Type 1 and Type 2 approaches.
   - In this method, you maintain a current value in the main dimension table (Type 1) while also creating new rows for historical changes (Type 2).
   - This provides both the most recent data and a history of changes, but it can be more complex to manage compared to other methods.
   - Example:

|   SurrogateID   | CustomerID |   Name   |   City       |    PrevCity      |      email        |    StartDate      |    EndDate      |   isActive   |   ......     |
|-----------------|------------|----------|--------------|------------------|-------------------|-------------------|-----------------|--------------|--------------|
|        1        |     1      |   Adam   |  Miami       |    NULL          |  adam@example.com |    01-Jan-2020    |   25-Mar-2020   |   False      |  ......      |
|        2        |     1      |   Adam   |  New Jersey  |   New York       |  adam@example.com |    25-Mar-2020    |   01-Dec-2020   |   False      |   ......      |
|        3        |     1      |   Adam   |  Miami       |   New Jersey     | adam@example.com  |    01-Dec-2020    |      NULL       |   True       | ......       |



| SCD Type | Summary |
|----------|---------|
| Type 0   | Ignore any changes and audit the changes.|
| Type 1   | Overwrite the changes |
| Type 2   | History will be added as a new row. |
| Type 3   | History will be added as a new column. |
| Type 4   | A new dimension will be added |
| Type 6   | Combination of Type 2 and Type 3 |

The choice of which SCD type to use depends on the specific needs of your organization and the nature of the data you are working with. It's essential to consider factors like data update frequency, the importance of historical data, query performance, and data storage constraints when deciding on the appropriate SCD method for your dimension modeling. Type 1 and Type 2 are the most commonly used SCD types, but the others may be valuable in certain scenarios.

Choosing the appropriate SCD method depends on the specific requirements of the organization and the nature of the data. Type 2 SCD is the most common choice for data warehousing and analytics, as it strikes a balance between historical preservation and simplicity. However, Type 1 and Type 3 can be useful in certain scenarios.

SCD is essential because it ensures the accuracy of data in data warehousing and analytics environments, enabling organizations to analyze historical trends, track changes, and make informed decisions based on historical data. The choice of SCD method should align with the business needs and the dimension modeling strategy in place.